<a href="https://colab.research.google.com/github/Up2Iso/label_selection_bias/blob/main/label_selection_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install aif360
!pip install pgmpy
!pip install fairlearn
!pip install BlackBoxAuditing

In [ ]:
%matplotlib inline
# Load all necessary packages

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from aif360.datasets import BinaryLabelDataset
import numpy as np
import random
import pandas as pd
import math
import matplotlib.pyplot as plt

from aif360.algorithms.preprocessing import Reweighing
from sklearn.ensemble import RandomForestClassifier

from aif360.algorithms.preprocessing.reweighing import Reweighing

from IPython.display import Markdown, display
from aif360.datasets import BinaryLabelDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


from aif360.algorithms.inprocessing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# inizialize random
np.random.seed(42)
random.seed(42)

pip install 'aif360[LawSchoolGPA]'


In [ ]:
# Universal Variables
n_databases = 10
discriminations = [0.1*ind for ind in range(10)]
disparity = 0
var_n = 20
size_train = 10000
size_test = 10000
threshold = 1
doubt = 0
label_names = 'Y'
favorable_classes = 1
sensitive_attribute = 'A'
privileged_groups = [{sensitive_attribute: 1}]
unprivileged_groups = [{sensitive_attribute: 0}]

In [ ]:
def eval_model(predictions, true_labels, sensitive_attribute):
    if len(predictions)!=len(true_labels) or len(predictions)!=len(sensitive_attribute):
        print('error: length of inputs must be equal')
        return False
    predictions = pd.Series(predictions)
    true_labels = pd.Series(true_labels)
    sensitive_attribute = pd.Series(sensitive_attribute)

    filtro = (sensitive_attribute == 1)
    a1 = predictions[filtro].sum()/(filtro.sum())

    filtro = (sensitive_attribute == 0)
    a0 = predictions[filtro].sum()/(filtro.sum())

    disparity = a1-a0

    accuracy = (predictions == true_labels).sum()/len(predictions)

    return [accuracy, disparity]

def calc_prob(dataframe, y, y_value, X=[], X_value=[]):
        '''calculates the conditional probability P(y=y_value| X=X_value) on the dataframe considered'''
        filtro = pd.Series([True for ind in range(dataframe.shape[0])])
        for ind in range(len(X)):
            filtro = filtro & dataframe[X[ind]] == X_value[ind]
        den = filtro.sum()
        filtro = filtro & dataframe[y] == y_value
        num = filtro.sum()
        return num/den if den != 0  else 1

def prob_G_given_A(disparity):
    '''returns [[P(G=0|A=0),P(G=0|A=1)],
                [P(G=1|A=0),P(G=1|A=1)] '''
    prob= (disparity+1)/2
    return [[prob, 1- prob],[1-prob, prob]]

def prob_keep(disparity, discrimination):
    '''returns [[P(not keep|A=0,G=0),P(not keep=0|A=0,G=1),P(not keep=0|A=1,G=0),P(not keep=0|A=1,G=1)],
                [P(keep|A=0,G=0),P(keep|A=0,G=1),P(keep|A=1,G=0),P(keep|A=1,G=1)] '''

    if disparity == 1:
        odds = [float('inf'), 0]
    elif disparity == -1:
        odds = [0, float('inf')]
    else:
        proba = prob_G_given_A(disparity)
        odds = [proba[0][0]/proba[1][0], proba[0][1]/proba[1][1]]

    #assuming P(keep|y=0,a)*P(keep|y=0,not a)/P(keep|y=1,a)P(keep|y=1,neg a) = 1
    q = disparity+discrimination
    a = (q-1)*odds[0]
    b = (odds[0]*odds[1]+1)*q
    c = odds[1]*(q+1)
    odd_keep = -b/(2*a) - ((b**2 - 4*a*c)**(0.5))/(2*a)
    odd_keep = [odd_keep, 1/odd_keep]
    #assuming P(keep|y=0,a=0) = 1 = P(keep|y=1,a=1)
    proba = [1, 1/odd_keep[0], odd_keep[1],1]
    proba = [[1-ind for ind in proba], proba]
    return proba

def prob_X_given_AG(threshold):
    '''returns [[P(X=0|A=0,G=0),P(X=0|A=0,G=1),P(X=0|A=1,G=0),P(X=0|A=1,G=1)],
                [P(X=1|A=0,G=0),P(X=1|A=0,G=1),P(X=1|A=1,G=0),P(X=1|A=1,G=1)] '''
    num_points = 4
    prob = [0.3, 0.4, 0.5, 0.6]
    random.shuffle(prob)
    return [prob, [1-prob[i] for i in range(num_points)]]

def prob_Y_given_AG(discrimination, disparity):
    """ P(Y=1| A=0,G=0) P(Y=1| A=0,G=1) P(Y=1| A=1,G=0) P(Y=1| A=1,G=1) """
    prob = [0,0,0,0]
    error = doubt
    prob[0] = error
    prob[3] = 1 - error
    #assuming P(y|a,not g) =  P(not y|not a, g)
    prob[2] =  (discrimination + error*(1 + disparity))/(1-disparity)
    prob[1] = 1-prob[2]
    return [[1-ind for ind in prob], prob]

def generate_model(threshold, disparity):
    cpd_X = [TabularCPD(variable=('X'+str(i)),
                       variable_card=2,
                       values=prob_X_given_AG(threshold),
                       evidence=[sensitive_attribute,'G'],
                       evidence_card=[2,2]) for i in range(var_n) ]
    cpd_Y = TabularCPD(variable=label_names,
                       variable_card=2,
                       values=prob_Y_given_AG(discrimination, disparity),
                       evidence=[sensitive_attribute,'fair_Y'],
                       evidence_card=[2,2])
    cpd_A = TabularCPD(variable=sensitive_attribute,
                       variable_card=2,
                       values=[[0.5], [0.5]])
    cpd_G = TabularCPD(variable='G',
                       variable_card=2,
                       values=prob_G_given_A(disparity),
                       evidence = [sensitive_attribute],
                       evidence_card = [2]
                       )
    cpd_keep = TabularCPD(variable='keep',
                       variable_card=2,
                       values=prob_keep(disparity, discrimination),
                       evidence = [sensitive_attribute, 'fair_Y'],
                       evidence_card = [2,2]
                       )
    cpd_fair = TabularCPD(variable='fair_Y',
                       variable_card=2,
                       values= [[1-doubt,doubt],[doubt,1-doubt]],
                       evidence = ['G'],
                       evidence_card = [2]
                       )
    stat_parity_edges = ([tuple([sensitive_attribute, 'X' + str(i)]) for i in range (var_n)]+
                         [tuple(['G', 'X' + str(i)]) for i in range (var_n)]+
                         [(sensitive_attribute, label_names),(sensitive_attribute, 'G'), ('fair_Y', label_names)]+
                         [('G','fair_Y')]+
                         [(sensitive_attribute, 'keep'), ('fair_Y', 'keep') ])

    stat_parity_bayes = BayesianNetwork(stat_parity_edges)
    stat_parity_bayes.add_cpds(*cpd_X,cpd_Y,cpd_A,cpd_G,cpd_keep,cpd_fair)
    stat_parity_bayes.check_model()
    return BayesianModelSampling(stat_parity_bayes).forward_sample

def generate_train(threshold, disparity):
    stat_parity_bayes = generate_model(threshold, disparity)

    df = stat_parity_bayes(size_train)
    train_dataset_label = df.drop(columns = ['G', 'keep','fair_Y'])

    new_size = int(size_train*size_train/ (df['keep'] == 1).sum())
    train_dataset_selection = stat_parity_bayes(new_size)
    filter = train_dataset_selection['keep'] == 1
    train_dataset_selection = train_dataset_selection[filter].drop(columns = ['keep',label_names,'G'])
    train_dataset_selection.rename(columns={'fair_Y': label_names},inplace=True)

    test_dataset_fair = stat_parity_bayes(size_test)
    test_dataset_fair.rename(columns={label_names:'unfair_Y', 'fair_Y': label_names},inplace=True)
    test_dataset_fair = test_dataset_fair.drop(columns = ['keep','G','unfair_Y'])

    return (train_dataset_label, train_dataset_selection, test_dataset_fair)

In [ ]:
def iter_model(iter_function, name, verbose = False, save = False):
    for ind,discrimination in enumerate(discriminations):
        avg_disparity_label = 0
        avg_accuracy_label = 0
        avg_disparity_selection = 0
        avg_accuracy_selection = 0
        for j in range(n_databases):
            train_dataset_label, train_dataset_selection, test_dataset_fair = label_bias_datasets[ind][j], selection_bias_datasets[ind][j], fair_datasets[ind][j]
            train_aif_label = BinaryLabelDataset(df=train_dataset_label, label_names = label_names, protected_attribute_names= sensitive_attribute)
            train_aif_selection = BinaryLabelDataset(df=train_dataset_selection, label_names = label_names, protected_attribute_names= sensitive_attribute)
            test_aif_fair = BinaryLabelDataset(df=test_dataset_fair, label_names = label_names, protected_attribute_names= sensitive_attribute)

            if name+'_label_accuracy' not in risultati.columns:
                risultati[name+'_label_accuracy'] = 0.0
                risultati[name+'_label_disparity'] = 0.0
                risultati[name+'_selection_accuracy'] = 0.0
                risultati[name+'_selection_disparity'] = 0.0

            mom_acc, mom_disp = iter_function(train_aif_label, test_aif_fair)
            avg_disparity_label += mom_disp
            avg_accuracy_label += mom_acc

            mom_acc, mom_disp = iter_function(train_aif_selection, test_aif_fair)
            avg_disparity_selection += mom_disp
            avg_accuracy_selection += mom_acc
            if verbose:
                print('dataset done: disparity=',discrimination,'number=',j)
        risultati.loc[discrimination][[name+'_label_accuracy', name+'_label_disparity']] = [avg_accuracy_label/n_databases, avg_disparity_label/n_databases]
        risultati.loc[discrimination][[name+'_selection_accuracy', name+'_selection_disparity']] = [avg_accuracy_selection/n_databases, avg_disparity_selection/n_databases]
        if save:
            with open('/content/gdrive/My Drive/risultati.csv', 'w') as f:
                f.write(risultati.to_csv())

In [ ]:
%%capture
label_bias_datasets = [[0 for j in range(n_databases)] for ind in discriminations]
selection_bias_datasets = [[0 for j in range(n_databases)] for ind in discriminations]
fair_datasets = [[0 for j in range(n_databases)] for ind in discriminations]
for ind,discrimination in enumerate(discriminations):
    for j in range(n_databases):
        label_bias_datasets[ind][j], selection_bias_datasets[ind][j], fair_datasets[ind][j] = generate_train(threshold,disparity)

risultati = pd.DataFrame(index = discriminations, dtype= 'float')

In [ ]:
np.random.seed(42)
random.seed(42)

def forest_result(aif_train, aif_test):

    forest_classifier = RandomForestClassifier()
    X_train = aif_train.features
    y_train = aif_train.labels.ravel()

    forest_classifier.fit(X_train, y_train)
    predictions = forest_classifier.predict(aif_test.features)
    return eval_model(predictions, aif_test.labels.ravel(), aif_test.protected_attributes.ravel() )

iter_model(forest_result, 'forest')

In [ ]:
with open('/content/gdrive/My Drive/risultati.csv', 'w') as f:
  f.write(risultati.to_csv())

In [ ]:
np.random.seed(42)
random.seed(42)

def reweighing_result(aif_train, aif_test):
    '''Data Preprocessing Techniques for Classification without Discrimination'''
    RW = Reweighing(unprivileged_groups=unprivileged_groups,
                    privileged_groups=privileged_groups)
    forest_classifier = RandomForestClassifier(random_state=42)
    dataset_aif_reweight_train = RW.fit_transform(aif_train)
    X_train = dataset_aif_reweight_train.features
    y_train = dataset_aif_reweight_train.labels.ravel()

    forest_classifier.fit(X_train, y_train, sample_weight=dataset_aif_reweight_train.instance_weights)
    predictions = forest_classifier.predict(aif_test.features)
    return eval_model(predictions, aif_test.labels.ravel(), aif_test.protected_attributes.ravel())

iter_model(reweighing_result, 'reweighing')

In [ ]:
with open('/content/gdrive/My Drive/risultati.csv', 'w') as f:
  f.write(risultati.to_csv())

In [ ]:
%%capture
np.random.seed(42)
random.seed(42)

def adversarial_result(aif_train, aif_test):
    sess = tf.Session()
    adversarial_label_unfair = AdversarialDebiasing(privileged_groups = privileged_groups,
                            unprivileged_groups = unprivileged_groups,
                            scope_name='adversarial_label_unfair',
                            adversary_loss_weight=0.5,
                            debias=True,
                            sess=sess)

    adversarial_label_unfair.fit(aif_train)
    predictions = adversarial_label_unfair.predict(aif_test).labels.ravel()

    sess.close()
    tf.reset_default_graph()
    return eval_model(predictions, aif_test.labels.ravel(), aif_test.protected_attributes.ravel())


iter_model(adversarial_result, 'adversarial')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
with open('/content/gdrive/My Drive/risultati.csv', 'w') as f:
  f.write(risultati.to_csv())

In [ ]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

np.random.seed(42)
random.seed(42)

def DIR_result(aif_train, aif_test):
    index = aif_train.feature_names.index(sensitive_attribute)

    di = DisparateImpactRemover(repair_level=1)
    train_repd = di.fit_transform(aif_train)
    test_repd = di.fit_transform(aif_test)

    X_tr = np.delete(train_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    X_te = np.delete(test_repd.features, index, axis=1)

    forest_classifier = RandomForestClassifier()

    forest_classifier.fit(X_tr, y_tr)
    predictions = forest_classifier.predict(X_te)
    return eval_model(predictions, aif_test.labels.ravel(), aif_test.protected_attributes.ravel())

iter_model(DIR_result, 'DIR')

In [ ]:
with open('/content/gdrive/My Drive/risultati.csv', 'w') as f:
  f.write(risultati.to_csv())

In [ ]:
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.algorithms.preprocessing.lfr_helpers import helpers as lfr_helpers

np.random.seed(42)
random.seed(42)

def LFR_results(aif_train, aif_test):
    scale_orig = StandardScaler()
    LFR_train = aif_train.copy(deepcopy= True)
    LFR_valid = aif_test.copy(deepcopy= True)

    LFR_train.features = scale_orig.fit_transform(LFR_train.features)
    LFR_valid.features = scale_orig.transform(LFR_valid.features)


    TR = LFR(unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups,
            k=10, Ax=0.1, Ay=1.0, Az=5
            )

    R = TR.fit(LFR_train, maxiter=5000, maxfun=5000)
    dataset_transf_valid = TR.transform(LFR_valid)
    predictions = dataset_transf_valid.labels.ravel()
    return eval_model(predictions, aif_test.labels.ravel(), aif_test.protected_attributes.ravel())

iter_model(LFR_results, 'LFR', verbose = True, save = True)

dataset done: disparity= 0.0 number= 0
dataset done: disparity= 0.0 number= 1
dataset done: disparity= 0.0 number= 2
dataset done: disparity= 0.0 number= 3
dataset done: disparity= 0.0 number= 4
dataset done: disparity= 0.0 number= 5
dataset done: disparity= 0.0 number= 6
dataset done: disparity= 0.0 number= 7
dataset done: disparity= 0.0 number= 8
dataset done: disparity= 0.0 number= 9
dataset done: disparity= 0.1 number= 0
dataset done: disparity= 0.1 number= 1
dataset done: disparity= 0.1 number= 2
dataset done: disparity= 0.1 number= 3
dataset done: disparity= 0.1 number= 4
dataset done: disparity= 0.1 number= 5
dataset done: disparity= 0.1 number= 6
dataset done: disparity= 0.1 number= 7
dataset done: disparity= 0.1 number= 8
dataset done: disparity= 0.1 number= 9
dataset done: disparity= 0.2 number= 0
dataset done: disparity= 0.2 number= 1
dataset done: disparity= 0.2 number= 2
dataset done: disparity= 0.2 number= 3
dataset done: disparity= 0.2 number= 4
dataset done: disparity= 

In [ ]:
with open('/content/gdrive/My Drive/risultati.csv', 'w') as f:
  f.write(risultati.to_csv())